<a href="https://colab.research.google.com/github/abinash00098/LEARNFLOW/blob/main/Climate_DAnalysis(Delhi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import files
uploaded = files.upload()


Saving DailyDelhiClimateTrain.csv to DailyDelhiClimateTrain (1).csv


In [3]:
# Load and prepare the data
data = pd.read_csv('DailyDelhiClimateTrain.csv')
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [4]:
# Data overview
print("Data Overview:")
print(data.head())
print("\nData Info:")
print(data.info())


Data Overview:
             meantemp   humidity  wind_speed  meanpressure
date                                                      
2013-01-01  10.000000  84.500000    0.000000   1015.666667
2013-01-02   7.400000  92.000000    2.980000   1017.800000
2013-01-03   7.166667  87.000000    4.633333   1018.666667
2013-01-04   8.666667  71.333333    1.233333   1017.166667
2013-01-05   6.000000  86.833333    3.700000   1016.500000

Data Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1462 entries, 2013-01-01 to 2017-01-01
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   meantemp      1462 non-null   float64
 1   humidity      1462 non-null   float64
 2   wind_speed    1462 non-null   float64
 3   meanpressure  1462 non-null   float64
dtypes: float64(4)
memory usage: 57.1 KB
None


In [5]:
# Visualize temperature trend
#Creates a plot showing the mean temperature trend over time.
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['meantemp'])
plt.title('Mean Temperature in Delhi Over Time')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.savefig('temperature_trend.png')
plt.close()

In [6]:
# Correlation heatmap -
# Generates a heatmap to visualize correlations between different climate variables.
correlation_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Climate Variables')
plt.savefig('correlation_matrix.png')
plt.close()


In [7]:

# Prepare data for modeling
data = data.fillna(method='ffill')

<ipython-input-7-3fe95d48917e>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


In [8]:
# Split the data
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]


In [9]:
# Build and train ARIMA model
#Prepares the data, splits it into training and testing sets, and builds an ARIMA model.
model = ARIMA(train['meantemp'], order=(5,1,0))
model_fit = model.fit()


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [10]:
# Make predictions
predictions = model_fit.forecast(steps=len(test))


In [11]:
# Visualize predictions
# Creates a plot showing the actual vs. predicted temperatures.
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['meantemp'], label='Train')
plt.plot(test.index, test['meantemp'], label='Test')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.title('Delhi Mean Temperature: Actual vs Predicted')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.savefig('temperature_forecast.png')
plt.close()

In [12]:
# Model evaluation
#Calculates and displays MAE and RMSE metrics.
mae = mean_absolute_error(test['meantemp'], predictions)
rmse = np.sqrt(mean_squared_error(test['meantemp'], predictions))
print(f'\nModel Evaluation:')
print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')



Model Evaluation:
MAE: 7.39
RMSE: 8.23


In [13]:
# Future predictions
# Generates and visualizes a 30-day forecast.
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=30)
future_predictions = model_fit.forecast(steps=30)


In [14]:

plt.figure(figsize=(12, 6))
plt.plot(data.index[-90:], data['meantemp'][-90:], label='Historical')
plt.plot(future_dates, future_predictions, label='Forecast', color='red')
plt.title('Delhi Mean Temperature: 30-Day Forecast')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.savefig('future_forecast.png')
plt.close()

In [15]:
# Save future predictions
future_df = pd.DataFrame({'Date': future_dates, 'Predicted_Mean_Temp': future_predictions})
future_df.to_csv('delhi_temperature_forecast.csv', index=False)

In [16]:
print("\nAnalysis complete. Results and visualizations have been saved.")


Analysis complete. Results and visualizations have been saved.
